<a href="https://colab.research.google.com/github/douglasmmachado/ExploratoryDataAnalysis/blob/master/Timeseries_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exploring the timeseries approach for the unified database

---



---



# Loading libs and database

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

raw_data_url ="https://raw.githubusercontent.com/douglasmmachado/ExploratoryDataAnalysis/master/project_database.csv"
df=pd.read_csv(raw_data_url, index_col=0)

categorical_features = ['ID_REF', 'ID_SITE_RATTACHE', 'CODE_ATC', 'HOSPI_CODE_UCD']
date_features = ['DATE_MOUV', 'WEEK', 'MONTH', 'YEAR']
numerical_features = ['N_UFS', 'QUANTITY', 'N_ETB', 'POPULATION', 'P_MEDICAL', 'PN_MEDICAL', 'LIT_HC', 'LIT_HP', 'SEJ_MCO', 'SEJ_HAD', 'SEJ_PSY', 'SEJ_SSR']

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75692 entries, 0 to 75691
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   ID_REF            75689 non-null  object 
 1   ID_SITE_RATTACHE  75690 non-null  object 
 2   CODE_ATC          75685 non-null  object 
 3   HOSPI_CODE_UCD    75690 non-null  object 
 4   DATE_MOUV         75685 non-null  object 
 5   N_UFS             75692 non-null  int64  
 6   QUANTITY          75685 non-null  float64
 7   WEEK              75685 non-null  float64
 8   MONTH             75692 non-null  int64  
 9   YEAR              75685 non-null  float64
 10  N_ETB             75692 non-null  int64  
 11  POPULATION        75692 non-null  float64
 12  P_MEDICAL         75692 non-null  int64  
 13  PN_MEDICAL        75692 non-null  int64  
 14  LIT_HC            75685 non-null  float64
 15  LIT_HP            75692 non-null  float64
 16  SEJ_MCO           75692 non-null  int64 

<ipython-input-15-af9092c990bd>:8: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv(raw_data_url, index_col=0)


# Data cleaning

In [16]:
df = df.drop_duplicates()
df = df.dropna()
df['DATE_MOUV'] = pd.to_datetime(df['DATE_MOUV'], 
               format='%Y-%m-%d')  # Changed the type of date too
               
for column in categorical_features:
  df[column] = df[column].astype('string')

df.sample(5)

<ipython-input-16-1c25a3c43dd3>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATE_MOUV'] = pd.to_datetime(df['DATE_MOUV'],
<ipython-input-16-1c25a3c43dd3>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].astype('string')


,ID_REF,ID_SITE_RATTACHE,CODE_ATC,HOSPI_CODE_UCD,DATE_MOUV,N_UFS,QUANTITY,WEEK,MONTH,YEAR,...,POPULATION,P_MEDICAL,PN_MEDICAL,LIT_HC,LIT_HP,SEJ_MCO,SEJ_HAD,SEJ_PSY,SEJ_SSR,SEJ_SLD
4576,800851,HOSPI_1,B05XA03,3400892761527,2017-07-11,4,920.0,28.0,7,2017.0,...,572443.0,806,5303,1360.0,106.0,80667,0,0,1239,122
14975,830631,HOSPI_1,N05BA12,3400892075761,2017-06-20,8,670.0,25.0,6,2017.0,...,572443.0,806,5303,1360.0,106.0,80667,0,0,1239,122
41580,503386,HOSPI_3,N02BE01,3400893875490,2016-09-02,15,780.0,35.0,9,2016.0,...,1132481.0,1290,7336,2030.0,506.0,120193,787,2755,1309,88
48087,504394,HOSPI_3,N05BA04,3400890837149,2014-05-29,1,10.0,22.0,5,2014.0,...,1107398.0,1158,7129,2063.0,521.0,117781,594,2903,1302,97
25604,9276152,HOSPI_2,B05XA03,3400892761527,2018-09-17,18,875.0,38.0,9,2018.0,...,541454.0,703,5007,1159.0,139.0,74663,0,1193,237,0


## Removing outliers by hospital and by drug

In [78]:
def remove_outliers(df, columns, verbose = False):
  N_macro = df.shape[0]
  print('N: ', N_macro)
  for col in columns:
      for id_ref in df['ID_REF'].unique():

          # Filter the dataframe for the specific ID_REF
          df_subset = df[df['ID_REF'] == id_ref]
          N = df_subset.shape[0]

          # Calculate statistics for the column within the ID_REF subset
          z_scores = np.abs(stats.zscore(df_subset[col]))
          threshold = 3

          # Remove outliers based on the z-score threshold within the ID_REF subset
          df_subset = df_subset[z_scores < threshold]
          N_new = df_subset.shape[0]
          delta = N - N_new

          # Update the original dataframe with the filtered values
          df = pd.concat([df[df['ID_REF'] != id_ref], df_subset])

          if verbose:
            if delta > 0:
                print(f'Column: {col}')
                print(f'ID_REF: {id_ref}')
                print(f'Removed outliers: {delta}')
                print(f'Removed percentage: {(100 * delta / N):.3}%')
                print()


  N_macro_new = df.shape[0]
  print('N_new: ', N_macro_new)
  delta = N_macro - N_macro_new
  print(f'Total removed outliers: {(100 * delta / N_macro):.3}%')
  return df

In [79]:
n_medicines = df['ID_REF'].nunique()
print('Number of different medicines: ', n_medicines)

df_h1 = df[df["ID_SITE_RATTACHE"] == "HOSPI_1"]
df_h2 = df[df["ID_SITE_RATTACHE"] == "HOSPI_2"]
df_h3 = df[df["ID_SITE_RATTACHE"] == "HOSPI_3"]
df_h4 = df[df["ID_SITE_RATTACHE"] == "HOSPI_4"]

print("-"*100)
df_h1 = remove_outliers(df_h1, ["QUANTITY"], verbose = False)

print("-"*100)
df_h2 = remove_outliers(df_h2, ["QUANTITY"], verbose = False)

print("-"*100)
df_h3 = remove_outliers(df_h3, ["QUANTITY"], verbose = False)

print("-"*100)
df_h4 = remove_outliers(df_h4, ["QUANTITY"], verbose = False)


Number of different medicines:  84
----------------------------------------------------------------------------------------------------
N:  22725
N_new:  22357
Total removed outliers: 1.62%
----------------------------------------------------------------------------------------------------
N:  15439
N_new:  15231
Total removed outliers: 1.35%
----------------------------------------------------------------------------------------------------
N:  27591
N_new:  27386
Total removed outliers: 0.743%
----------------------------------------------------------------------------------------------------
N:  9929
N_new:  9808
Total removed outliers: 1.22%
